In [1]:
%%sh
pwd

/mnt/m/Documents/DNN Final Project/spacecraft-pose-pose-estimation-runtime/juptyer-notebooks


In [1]:
import os
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation
from sklearn.model_selection import train_test_split
import keras
import keras.preprocessing.image as kimg
from keras.preprocessing import image_dataset_from_directory
from keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, Concatenate, Flatten, Conv2D, MaxPooling2D, AvgPool2D
from keras.applications import ResNet50, MobileNetV2
from keras.optimizers import Adam
from keras import Model
import math
from math import cos, sin
from autograd import jacobian
import tensorflow as tf
#import tensorflow_probability as tfp
#tfd = tfp.distributions
from tensorflow.data import Dataset
from tensorflow.keras.utils import image_dataset_from_directory as idfd
from keras.models import Sequential
import keras.backend as K

2024-04-24 22:09:11.997610: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-24 22:09:11.997679: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-24 22:09:12.012743: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-24 22:09:12.069174: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-24 22:09:13.978594: W tensorflow/compiler/tf2

In [2]:
root = '/mnt/m/Documents/DNN Final Project/spacecraft-pose-pose-estimation-runtime/juptyer-notebooks'

In [3]:
CAMERA_MATRIX = np.array([[5.2125371e+03, 0.0000000e+00, 6.4000000e+02],
                          [0.0000000e+00, 6.2550444e+03, 5.1200000e+02],
                          [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]])
IMAGE_WIDTH = 1280
IMAGE_HEIGHT = 1024

In [2]:
def load_and_preprocess_image(path, base_path):
  raw = tf.io.read_file(path)
  image = tf.image.resize(tf.image.decode_png(raw, channels=3), (512,640))
  base_raw = tf.io.read_file(base_path)
  base_image = tf.image.resize(tf.image.decode_png(base_raw, channels=3), (512,640))

  return base_image, image


#df_labels = dfY[["x","y","z","qw","qx","qy","qz"]]
#dfY.drop(["chain_id", "i", "x","y","z","qw","qx","qy","qz"], axis=1, inplace=True)
#print(dfY)
#dfY = np.asarray(dfY).astype(np.float32)
#dfY.drop(["chain_id", "i"], axis=1, inplace=True)
#print(dfY)
#ds = tf.data.Dataset.from_tensor_slices((dfY, {"x": "x", "y": "y", "z": "z", "qw": "qw", "qx": "qx", "qy": "qy", "qz": "qz"}))

# The tuples are unpacked into the positional arguments of the mapped function
def load_and_preprocess_from_path_label(path, base_path, label):
  return load_and_preprocess_image(path, base_path), label

# train_images = x_train.apply(lambda x: load_and_preprocess_image(x["path"],x["base_path"]), axis=1)
# test_images = x_test.apply(lambda x: load_and_preprocess_image(x["path"],x["base_path"]), axis=1)
# # print(type(image_label_ds[0]))
# # print(image_label_ds[:,0])
# base_images = [x[0] for x in train_images]
# images = [x[1] for x in train_images]
# base_images_test = [x[0] for x in test_images]
# images_test = [x[1] for x in test_images]
# print(np.asarray(base_images).shape)
# for elem in image_label_ds.take(1):
#   print(elem[0][0].shape)
#print(list(image_label_ds.as_numpy_iterator())[0][0])

In [4]:
def flow_from_df(dataframe: pd.DataFrame, chunk_size: int = 10):
    for start_row in range(0, dataframe.shape[0], chunk_size):
        end_row  = min(start_row + chunk_size, dataframe.shape[0])
        yield dataframe.iloc[start_row:end_row, :]
get_chunk = flow_from_df(x_train, 100)

def load_new_batch():
    train_images = next(get_chunk).apply(lambda x: load_and_preprocess_image(x["path"],x["base_path"]), axis=1)
    base_images = [x[0] for x in train_images]
    images = [x[1] for x in train_images]
    del train_images

    return base_images, images

In [8]:
dfY = pd.read_csv("../data/train_labels.csv")[:15000]
dfY["path"] = [f"../data/images/{id}/{i:03d}.png" for id, i in zip(dfY["chain_id"], dfY["i"])]
dfY["base_path"] = [f"../data/images/{id}/000.png" for id, i in zip(dfY["chain_id"], dfY["i"])]
x_train, x_test, y_train, y_test = train_test_split(dfY[["path", "base_path"]], dfY[["x", "y", "z", "qw","qx","qy","qz"]], test_size=0.95, random_state=42)
x_train = pd.DataFrame(x_train, columns=["path", "base_path"])
x_test = pd.DataFrame(x_test, columns=["path", "base_path"])

In [9]:

#ypred_rot = pd.DataFrame(model_rot.predict(CustomDataGenerator(x_test, y_test, 20)), index=y_test.index, columns=["qw", "qx", "qy", "qz"])
ypred = pd.DataFrame(model_pos.predict(CustomDataGenerator(x_test, y_test, 50)), index=y_test.index, columns=["x", "y", "z", "qw", "qx", "qy", "qz"])
#ypred = pd.merge(ypred_pos, ypred_rot, left_index=True, right_index=True)
#ypred[["qw", "qx", "qy", "qz"]] = ypred[["qw", "qx", "qy", "qz"]].clip(lower=-1, upper=1)

ypred["chain_id"] = dfY.iloc[ypred.index].chain_id
ypred["i"] = dfY.iloc[ypred.index].i.astype(int)
y_pred_norm = np.linalg.norm(ypred[["qw", "qx", "qy", "qz"]], axis=1)
ypred["qw"] = ypred["qw"] / y_pred_norm
ypred["qx"] = ypred["qx"] / y_pred_norm
ypred["qy"] = ypred["qy"] / y_pred_norm
ypred["qz"] = ypred["qz"] / y_pred_norm
#ypred.loc[ypred.i == 0, ["x", "y", "z", "qw", "qx", "qy", "qz"]] = [0,0,0,1,0,0,0]
# ypred[["qw", "qx", "qy", "qz"]] = [0,0,0,0]
#ypred[["x", "y", "z"]] = [0,0,0]
#ypred = ypred[["chain_id", "i", "x", "y", "z", "qw", "qx", "qy", "qz"]]
ypred.loc[ypred.i == 0, ["x", "y", "z", "qw", "qx", "qy", "qz"]] = [0,0,0,1,0,0,0]
ypred.set_index(["chain_id", "i"], inplace=True)
ToCSV(ypred, "cnn-test")
y_copy = y_test.copy()
y_copy["chain_id"] = dfY.iloc[y_copy.index].chain_id
y_copy["i"] = dfY.iloc[y_copy.index].i.astype(int)
y_copy.set_index(["chain_id", "i"], inplace=True)
ToCSV(y_copy, "cnn-true")

#model.save('../models/test_model.keras')

Found 14250 images belonging to None classes
285/285 [==============================] - 1229s 4s/step


In [7]:
model_pos.save('../models/test_model-logcosh-16x3-3e-00001-larger-kernels.keras')

In [3]:
ToCSV = lambda df, fname: df.round(2).to_csv(f'{fname}.csv')

# Loads all images found and trains given model
def load_images_and_train_model(model, is_pos_model):
    with tf.device('/GPU:0'):
        model.summary()
        
        #for i in range(batches):
            #batch = load_new_batch()
            #model.fit([np.asarray(batch[0]), np.asarray(batch[1])], y_train[100 * i : 100*(i+1)], epochs=40, batch_size=batch_size_b)
        if is_pos_model:
            ypos = pd.DataFrame(y_train, index=y_train.index, columns=["x", "y", "z"])
            model.fit(CustomDataGenerator(x_train, y_train, batch_size=100), epochs=3)
        else:
            yquat = pd.DataFrame(y_train, index=y_train.index, columns=["qw", "qx", "qy", "qz"])
            model.fit(CustomDataGenerator(x_train, yquat, batch_size=100), epochs=3)


        
    #     ypreddf = pd.DataFrame(ypred.T, index=image_label_ds.take(count * 100 * 0.2).index_array, columns=["x", "y", "z"])
    #     dfY = pd.read_csv("../data/train_labels.csv")[:count * 100]
    #     dfY_merged = pd.merge(ypreddf.drop(["x", "y", "z"], axis=1), dfY, left_index=True, right_index=True)
    #     dfYidx = dfY.drop(["x", "y", "z", "qw", "qx", "qy", "qz"], axis=1)
    #     df_merged = pd.merge(ypreddf, dfYidx, left_index=True, right_index=True)
    #     df_merged["qw"] = 0
    #     df_merged["qx"] = 0
    #     df_merged["qy"] = 0
    #     df_merged["qz"] = 0
    #     df_merged.loc[df_merged["i"] == 0, ["x", "y", "z", "qw", "qx", "qy", "qz"]] = [0,0,0,1,0,0,0]
    #     df_merged.set_index(["chain_id", "i"], inplace=True)
    #    # df_merged.drop(["id"], axis=1, inplace=True)
    #     ToCSV(df_merged, "cnn-test")
    #     ToCSV(dfY_merged, "cnn-true")

    return model

# from https://github.com/1988kramer/camera-pose/blob/master/camera-pose.py 
def create_conv_branch(input_shape):
	model = Sequential()
	model.add(Conv2D(96, kernel_size=(11,11),
					 strides=4, padding='valid',
					 activation='relu',
					 input_shape=input_shape))
	model.add(MaxPooling2D(pool_size=(2,2), strides=2))
	model.add(Conv2D(128, kernel_size=(5,5),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(MaxPooling2D(pool_size=(3,3), strides=1))
	model.add(Conv2D(256, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(256, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(128, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	# model.add(MaxPooling2D(pool_size=(3,3), strides=2))
	# model.add(Conv2D(256, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	# model.add(Conv2D(128, kernel_size=(3,3),
	# 				 strides=1, padding='same',
	# 				 activation='relu'))
	return model

def create_conv_branch_pos(input_shape):
	model = Sequential()
	model.add(Conv2D(96, kernel_size=(11,11),
					 strides=4, padding='valid',
					 activation='relu',
					 input_shape=input_shape))
	model.add(MaxPooling2D(pool_size=(2,2), strides=2))
	model.add(Conv2D(128, kernel_size=(7,7),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(MaxPooling2D(pool_size=(3,3), strides=1))
	model.add(Conv2D(256, kernel_size=(5,5),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(256, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(128, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(MaxPooling2D(pool_size=(3,3), strides=2))
	model.add(Conv2D(256, kernel_size=(5,5),
					 strides=1, padding='same',
					 activation='relu'))
	model.add(Conv2D(128, kernel_size=(3,3),
					 strides=1, padding='same',
					 activation='relu'))
	return model

In [4]:
# https://stackoverflow.com/questions/63827339/how-to-build-a-custom-data-generator-for-keras-tf-keras-where-x-images-are-being
class CustomDataGenerator(tf.keras.utils.Sequence):

    ''' Custom DataGenerator to load img 
    
    Arguments:
        data_frame = pandas data frame in filenames and labels format
        batch_size = divide data in batches
        shuffle = shuffle data before loading
        img_shape = image shape in (h, w, d) format
        augmentation = data augmentation to make model rebust to overfitting
    
    Output:
        Img: numpy array of image
        label : output label for image
    '''
    
    def __init__(self, data_frame, labels, batch_size=10, img_shape=None, augmentation=True, num_classes=None):
        self.data_frame = data_frame
        self.labels = labels
        self.train_len = len(data_frame)
        self.batch_size = batch_size
        self.img_shape = img_shape
        self.num_classes = num_classes
        print(f"Found {self.data_frame.shape[0]} images belonging to {self.num_classes} classes")

    def __len__(self):
        ''' return total number of batches '''
        #self.data_frame = shuffle(self.data_frame)
        return math.ceil(self.train_len/self.batch_size)

    def on_epoch_end(self):
        ''' shuffle data after every epoch '''
        # fix on epoch end it's not working, adding shuffle in len for alternative
        pass
    
    def __data_augmentation(self, img):
        ''' function for apply some data augmentation '''
        img = tf.image.resize(img, 512, 640)
        img = tf.image.random_flip_up_down(img)
        return img
        
    def __get_image(self, file_id):
        """ open image with file_id path and apply data augmentation """
        img = np.asarray(Image.open(file_id))
        img = np.resize(img, self.img_shape)
        img = self.__data_augmentation(img)

        return img

    def __get_label(self, label_id):
        """ uncomment the below line to convert label into categorical format """
        #label_id = tf.keras.utils.to_categorical(label_id, num_classes)
        return label_id

    def __getitem__(self, idx):
        # batch_x = self.data_frame["filenames"][idx * self.batch_size:(idx + 1) * self.batch_size]
        # batch_y = self.data_frame["labels"][idx * self.batch_size:(idx + 1) * self.batch_size]
        # # read your data here using the batch lists, batch_x and batch_y
        # x = [self.__get_image(file_id) for file_id in batch_x] 
        # y = [self.__get_label(label_id) for label_id in batch_y]

        y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        paths = self.data_frame["path"][idx * self.batch_size:(idx + 1) * self.batch_size]
        base_paths = self.data_frame["base_path"][idx * self.batch_size:(idx + 1) * self.batch_size]

        #raw = [tf.io.read_file(path) for path in paths]
        image = [tf.image.resize(tf.image.decode_png(tf.io.read_file(path), channels=3), (512,640)) for path in paths]
        #base_raw = tf.io.read_file(base_path)
        base_image = [tf.image.resize(tf.image.decode_png(tf.io.read_file(path), channels=3), (512,640)) for path in base_paths]

        return [np.asarray(base_image), np.asarray(image)], y

        #return tf.convert_to_tensor(x), tf.convert_to_tensor(y)

In [5]:

input_shape=(512, 640, 3)

# Rotation model
# conv_branch = create_conv_branch(input_shape)
# branch_a = Input(shape=input_shape)
# branch_b = Input(shape=input_shape)

# processed_a = conv_branch(branch_a)
# processed_b = conv_branch(branch_b)

# concat = keras.layers.concatenate([processed_a, processed_b])
# x1 = Flatten()(concat)
# output = Dense(4)(x1)

# model_rot = Model(inputs=[branch_a,branch_b], outputs=output)
# model_rot.compile(optimizer=Adam(learning_rate=0.0025), loss="mae")
# model_rot.summary()
# load_images_and_train_model(model_rot, False)

# Position model
conv_branch = create_conv_branch_pos(input_shape)
branch_a = Input(shape=input_shape)
branch_b = Input(shape=input_shape)

processed_a = conv_branch(branch_a)
processed_b = conv_branch(branch_b)

concat = keras.layers.concatenate([processed_a, processed_b])

x1 = Flatten()(concat)
x1 = Dropout(0.25)(x1)
x1 = Dense(16)(x1)
x1 = Dropout(0.25)(x1)
x1 = Dense(16)(x1)
x1 = Dropout(0.25)(x1)
x1 = Dense(16)(x1)
output = Dense(7)(x1)

model_pos = Model(inputs=[branch_a,branch_b], outputs=output)
model_pos.compile(optimizer=Adam(learning_rate=0.0001), loss="log_cosh")
model_pos.summary()
load_images_and_train_model(model_pos, True)

# yhat = model(x_test)
# var_inv = 1 / yhat.variance()

2024-04-24 22:09:21.403642: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-24 22:09:21.626639: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-24 22:09:21.626957: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-24 22:09:21.629304: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-24 22:09:21.629572: I external/local_xla/xla/stream_executor

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 640, 3)]        0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 512, 640, 3)]        0         []                            
                                                                                                  
 sequential (Sequential)     (None, 30, 38, 128)          3456256   ['input_1[0][0]',             
                                                                     'input_2[0][0]']             
                                                                                                  
 concatenate (Concatenate)   (None, 30, 38, 256)          0         ['sequential[0][0]',      

2024-04-24 22:09:42.780508: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-04-24 22:09:43.215798: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-04-24 22:09:47.484856: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.71GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-24 22:09:47.679924: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.71GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-04-24 22:09:51.751032: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran o

19/19 [==============================] - 280s 11s/step - loss: 58.7322
Epoch 2/3
19/19 [==============================] - 168s 9s/step - loss: 58.5057
Epoch 3/3
19/19 [==============================] - 161s 8s/step - loss: 58.3347


In [ ]:
for i in 

In [59]:
# Correspondences encode - encode to d=128
# Stack correspondences (x^1_i, x^2_i) to x^12 in R^(nx4)
stackedCorr = [[a[0][0], a[0][1], b[0][0], b[0][1]] for (a,b) in zip(points1, points2)]
#print(stackedCorr)

m = keras.models.Sequential([
    Input(shape=(None, 4)),
    Dense(128, activation="relu")# Embed correspondences to d=128
    # Create graph CNN
    # Concat to ResNet
])
m.summary()
m.compile()
m.fit()

2024-03-25 23:45:46.499293: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-25 23:45:46.777884: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-25 23:45:46.778160: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-25 23:45:46.780087: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-25 23:45:46.780382: I external/local_xla/xla/stream_executor

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, None, 128)         640       
                                                                 
Total params: 640 (2.50 KB)
Trainable params: 640 (2.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
def calculate_pose(homography_matrix):
    # Extract rotation and translation from homography matrix
    rot_matrix = np.zeros((3, 3))
    rot_matrix[:, :2] = homography_matrix[:, :2]
    rot_matrix[:, 2] = np.cross(rot_matrix[:, 0], rot_matrix[:, 1])
    translation = homography_matrix[:, 2] / np.linalg.norm(homography_matrix[:, :2])

    # Convert rotation matrix to quaternion
    trace = np.trace(rot_matrix)
    if trace > 0:
        s = 0.5 / np.sqrt(trace + 1.0)
        w = 0.25 / s
        x = (rot_matrix[2, 1] - rot_matrix[1, 2]) * s
        y = (rot_matrix[0, 2] - rot_matrix[2, 0]) * s
        z = (rot_matrix[1, 0] - rot_matrix[0, 1]) * s
    else:
        if rot_matrix[0, 0] > rot_matrix[1, 1] and rot_matrix[0, 0] > rot_matrix[2, 2]:
            s = 2.0 * np.sqrt(1.0 + rot_matrix[0, 0] - rot_matrix[1, 1] - rot_matrix[2, 2])
            w = (rot_matrix[2, 1] - rot_matrix[1, 2]) / s
            x = 0.25 * s
            y = (rot_matrix[0, 1] + rot_matrix[1, 0]) / s
            z = (rot_matrix[0, 2] + rot_matrix[2, 0]) / s
        elif rot_matrix[1, 1] > rot_matrix[2, 2]:
            s = 2.0 * np.sqrt(1.0 + rot_matrix[1, 1] - rot_matrix[0, 0] - rot_matrix[2, 2])
            w = (rot_matrix[0, 2] - rot_matrix[2, 0]) / s
            x = (rot_matrix[0, 1] + rot_matrix[1, 0]) / s
            y = 0.25 * s
            z = (rot_matrix[1, 2] + rot_matrix[2, 1]) / s
        else:
            s = 2.0 * np.sqrt(1.0 + rot_matrix[2, 2] - rot_matrix[0, 0] - rot_matrix[1, 1])
            w = (rot_matrix[1, 0] - rot_matrix[0, 1]) / s
            x = (rot_matrix[0, 2] + rot_matrix[2, 0]) / s
            y = (rot_matrix[1, 2] + rot_matrix[2, 1]) / s
            z = 0.25 * s

    return translation, [x, y, z, w]


In [36]:
translation, quat = calculate_pose(M)

In [37]:
translation

array([620.71499515, 508.70476913,   0.92980055])

In [38]:
quat

[-0.00029664116316386924,
 -0.00031669406374587117,
 0.719521014529947,
 0.049691004139257325]

In [50]:
def move_image(image_path, pose):
    # Read the image
    image = cv.imread(image_path)
    
    # Extract translation and rotation from the pose
    translation = pose[:3]
    quaternion = pose[3:]
    
    # Convert quaternion to rotation matrix
    rotation = Rotation.from_quat(quaternion).as_matrix()
    
    # Apply translation
    translation_matrix = np.eye(4)
    translation_matrix[:3, 3] = translation
    # Apply rotation
    transformation_matrix = np.eye(4)
    transformation_matrix[:3, :3] = rotation
    
    # Apply transformation to image
    rows, cols, _ = image.shape
    new_image = cv.warpPerspective(image, transformation_matrix[:3], (cols, rows))
    
    return new_image

In [54]:
# Move the image
pose = [7.64022827,-46.98920059,-42.21747971,0.98997474,-0.0941886,-0.07856753,0.07003857]
# new_image = move_image(img1path, pose)

# # Show the original and moved image
# cv.imshow('Original Image', cv.imread(img1path))
# cv.imshow('Moved Image', new_image)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [113]:
image = cv.imread(img1path)
translation = np.float32(pose[:3])
quaternion = np.float32(pose[3:])
rotation = Rotation.from_quat(quaternion).as_matrix().astype(np.float32)

In [114]:
quaternion
print(type(quaternion))

<class 'numpy.ndarray'>


In [85]:
def quaternion_to_rotation_matrix(q):
    # Extract quaternion components
    qw, qx, qy, qz = q
    # Compute rotation matrix
    R = np.array([[1 - 2*qy**2 - 2*qz**2, 2*qx*qy - 2*qz*qw, 2*qx*qz + 2*qy*qw],
                  [2*qx*qy + 2*qz*qw, 1 - 2*qx**2 - 2*qz**2, 2*qy*qz - 2*qx*qw],
                  [2*qx*qz - 2*qy*qw, 2*qy*qz + 2*qx*qw, 1 - 2*qx**2 - 2*qy**2]])
    return R

In [88]:
R = quaternion_to_rotation_matrix(quaternion)
transformation_matrix = np.identity(4)
transformation_matrix[:3, :3] = R
transformation_matrix[:3, 3] = translation

In [115]:
rotation.dtype

dtype('float32')

In [117]:
cv.warpAffine(image, rotation, (1280, 1024))

error: OpenCV(4.9.0) /home/conda/feedstock_root/build_artifacts/libopencv_1704864625327/work/modules/imgproc/src/imgwarp.cpp:2757: error: (-215:Assertion failed) (M0.type() == CV_32F || M0.type() == CV_64F) && M0.rows == 2 && M0.cols == 3 in function 'warpAffine'


In [97]:
transformation_matrix

array([[  0.97784348,  -0.1238725 ,  -0.16875341,   7.64022827],
       [  0.15347316,   0.97244621,   0.17548315, -46.98920059],
       [  0.14236607,  -0.19749418,   0.9699113 , -42.21747971],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

In [93]:
# Normalize transformation matrix
transformation_matrix = transformation_matrix / transformation_matrix[3, 3]

# Extract rotation and translation components
rotation_matrix = transformation_matrix[0:3, 0:3]
translation_vector = transformation_matrix[0:3, 3]

projection_matrix = np.hstack([rotation_matrix, translation_vector.reshape(3, 1)])

In [96]:
transformed_image = cv.warpPerspective(image, projection_matrix, (image.shape[1], image.shape[0]))

error: OpenCV(4.9.0) /home/conda/feedstock_root/build_artifacts/libopencv_1704864625327/work/modules/imgproc/src/imgwarp.cpp:3323: error: (-215:Assertion failed) (M0.type() == CV_32F || M0.type() == CV_64F) && M0.rows == 3 && M0.cols == 3 in function 'warpPerspective'


In [91]:
transformed_image = cv.warpPerspective(image, transformation_matrix, (image.shape[1], image.shape[0]))

error: OpenCV(4.9.0) /home/conda/feedstock_root/build_artifacts/libopencv_1704864625327/work/modules/imgproc/src/imgwarp.cpp:3323: error: (-215:Assertion failed) (M0.type() == CV_32F || M0.type() == CV_64F) && M0.rows == 3 && M0.cols == 3 in function 'warpPerspective'


In [65]:
# Combine translation and rotation into transformation matrix
transformation_matrix = np.eye(4)
transformation_matrix[:3, :3] = rotation
transformation_matrix[:3, 3] = translation

In [67]:
rows, cols, _ = image.shape

In [69]:
new_image = cv.warpPerspective(image, transformation_matrix[:3], (cols, rows))

error: OpenCV(4.9.0) /home/conda/feedstock_root/build_artifacts/libopencv_1704864625327/work/modules/imgproc/src/imgwarp.cpp:3323: error: (-215:Assertion failed) (M0.type() == CV_32F || M0.type() == CV_64F) && M0.rows == 3 && M0.cols == 3 in function 'warpPerspective'
